In [ ]:
import requests

In [ ]:
def team_info(team_id: int, season: int, sport: str):
    if sport not in ('xc','tfo','tfi'): return 'error'
    td_sport = 'tf' if sport == 'tfo' else sport
    team_data = requests.get('https://www.athletic.net/api/v1/TeamNav/Team',params=dict(team=team_id,sport=td_sport,season=season))

    team_core = requests.get('https://www.athletic.net/api/v1/TeamHome/GetTeamCore',dict(teamId=team_id,sport=sport,season=season))
    anettokens = team_core.json()['jwtTeamHome']
    referer_url = f'https://www.athletic.net/team/{team_id}/cross-country/{season}'
    headers = {'referer':referer_url,'anettokens':anettokens}
    roster = requests.get('https://www.athletic.net/api/v1/TeamHome/GetAthletes',headers = headers, params = dict(seasonID = season))
    team_info = team_data.json()['team']

    schedule = requests.get('https://www.athletic.net/api/v1/TeamHomeCal/GetCalendar',headers = headers, params = dict(seasonID = season))
    schedule_output = [
        {
            'meet_name':meet['Name'],
            'anet_meet_id': meet['MeetID'], 
            'street_address':meet['StreetAddress'],
            'city': meet['City'], 
            'state': meet['State'],
            'zipcode': meet['PostalCode'],
            'location': meet['Location'],
            'date': meet['Date']
            
        } 
        for meet in schedule.json()
    ]
    return {
                'name' : team_info['Name'],
                'city' : team_info['City'],
                'state' : team_info['State'],
                'mascot' : team_info['Mascot'],
                'roster' : roster.json(),
                'schedule' : schedule_output#.json()
            }

team_info(493,2023,'xc')

In [ ]:
def get_meet_results(meet_id: int, sport: str):
    params = dict(meetId=meet_id, sport=sport)
    meet = requests.get('https://www.athletic.net/api/v1/Meet/GetMeetData',params=params)
    
    meet_payload = meet.json()
    
    meet_data = {
        'meet' :
            {
                'location' : meet_payload['meet']['Location']['Name'],
                'street_address' : meet_payload['meet']['Location']['Address'],
                'city' : meet_payload['meet']['Location']['City'],
                'state' : meet_payload['meet']['Location']['State'],
                'zipcode' : meet_payload['meet']['Location']['PostalCode']
            },
        'races': [
                    {
                        'race_id': race['IDMeetDiv'],
                        'race' : race['DivName'],
                        'division' : race['Division'],
                        'place_depth' : race['PlaceDepth'],
                        'score_depth' : race['ScoreDepth'],
                        'start_time' : race['RaceTime'],
                        'results' : [],
                        'team_scores' : []
                    }   
                    for race in meet_payload['xcDivisions']
        ]
    }
    
    results = requests.get('https://www.athletic.net/api/v1/Meet/GetAllResultsData', headers = dict(referer = 'https://www.athletic.net/CrossCountry/meet/225886/results/all',anettokens = meet_payload['jwtMeet']))
    
    for finisher in results.json()['results']:
        finisher_details = {
            'athlete_id' : finisher['AthleteID'],
            'name' : finisher['FirstName'] + ' ' + finisher['LastName'],
            'team_id' : finisher['TeamID'],
            'team' : finisher['SchoolName'],
            'grade' : finisher['AgeGrade'],
            'result' : finisher['Result'],
            'place' : finisher['Place']
        }
        race_id = finisher['RaceDivisionID']
        for race in meet_data['races']:
            if race_id == race['race_id']:
                race['results'].append(finisher_details)
                
    for team_score in results.json()['teamScores']:
        
        
    return meet_data
        

In [ ]:
r2.json()

In [ ]:
r4 = requests.get('https://www.athletic.net/api/v1/SignedInUser/GetSignedInUser',headers={'referer':'https://www.athletic.net/team/123/cross-country/2023'})

In [ ]:
r4.json()

In [1]:
from typing import Optional, List

from sqlmodel import Field, Relationship, Session, SQLModel, create_engine

from datetime import date, time

In [2]:
# class Athlete(SQLModel, table=True):
#     id: Optional[int] = Field(default=None, primary_key=True)
#     anet_id: int
#     name: str
#     gender: str

#     results: List['Result'] = Relationship(back_populates = 'athlete')

# class Team(SQLModel, table=True):
#     id: Optional[int] = Field(default=None, primary_key=True)
#     anet_id: int
#     name: str

# class Meet(SQLModel, table=True):
#     id: Optional[int] = Field(default=None, primary_key=True)
#     anet_id: int
#     name: str
#     location: str
#     date: date

# class Result(SQLModel, table=True):
#     id: Optional[int] = Field(default=None, primary_key=True)
#     anet_id: int
#     result: time
#     place: int

#     athlete_id: Optional[int] = Field(default=None, foreign_key = 'athlete.id')
#     athlete: Optional[Athlete] = Relationship(back_populates = 'results')
    
#     team_id: Optional[int] = Field(default=None, foreign_key = 'team.id')
#     meet_id: Optional[int] = Field(default=None, foreign_key = 'meet.id')

class SimpleResult(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    athlete_id: int
    result: time
    meet_id: int
    season: int
    school_id: int
    distance: int
    pr: bool
    sb: bool
    place: int


db_name = "anet_results"
db_string = f"postgresql:///{db_name}"

engine = create_engine(db_string, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

In [3]:
create_db_and_tables()

2023-09-23 23:12:15,057 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-09-23 23:12:15,058 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-23 23:12:15,060 INFO sqlalchemy.engine.Engine select current_schema()
2023-09-23 23:12:15,061 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-23 23:12:15,062 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-09-23 23:12:15,063 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-23 23:12:15,064 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-23 23:12:15,065 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-09-23 23:12:15,066 INFO sqlalchemy.engine.Engine [generated in 0.00084s] {'name': 'simpleresult'}
2023-09-23 23:12:15,068 INFO sqlalchemy.engine.Engine 
CREATE TABLE simpleresult (
	id SERIAL NOT NULL, 
	athlete_id INTEGER NOT NULL, 
	result TIME WITHOUT TIME ZONE NOT NULL, 
	meet_

In [ ]:
session = Session(engine)

for result in r2.json():
    athlete = Athlete(anet_id = result['ID'],name = result['Name'], gender = result['Gender'])
    session.add(athlete)

session.commit()

In [ ]:
session.close()

In [ ]:
ath = requests.get('https://www.athletic.net/api/v1/AthleteBio/GetAthleteBioData?athleteId=17657897&sport=xc&level=null')

In [ ]:
res = ath.json()['resultsXC'][0]

In [ ]:
session = Session(engine)
for r in ath.json()['resultsXC']:
    sr = SimpleResult(
        athlete_id = r['AthleteID'],
        result = r['Result'],
        place = r['Place'],
        pr = r['PersonalBest'],
        sb = r['SeasonBest'],
        school_id = r['SchoolID'],
        distance = r['Distance'],
        meet_id = r['MeetID'],
        season = r['SeasonID']
    )
    session.add(sr)
session.commit()
session.close()